<a href="https://colab.research.google.com/github/theSalted/DeepLearningNB/blob/main/TextGeneration/mini_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MiniGPT

In [1]:
import os

os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

In [2]:
import keras
import pathlib

extract_dir = keras.utils.get_file(
    fname="mini-c4",
    origin=(
        "https://hf.co/datasets/mattdangerw/mini-c4/resolve/main/mini-c4.zip"
    ),
    extract=True,
)
extract_dir = pathlib.Path(extract_dir) / "mini-c4"

1505728294/1505728294 ━━━━━━━━━━━━━━━━━━━━ 10s 0us/step


In [3]:
with open(extract_dir / "shard0.txt", "r") as f:
    print(f.readline().replace("\\n", "\n")[:100])

Beginners BBQ Class Taking Place in Missoula!
Do you want to get better at making delicious BBQ? You


In [4]:
import keras_hub
import numpy as np

vocabulary_file = keras.utils.get_file(
    origin="https://hf.co/mattdangerw/spiece/resolve/main/vocabulary.proto",
)

499723/499723 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
tokenizer = keras_hub.tokenizers.SentencePieceTokenizer(vocabulary_file)

In [6]:
tokenizer.tokenize("The quick bron fox.")

<tf.Tensor: shape=(6,), dtype=int32, numpy=array([  450,  4996, 12246,  1701, 29916, 29889], dtype=int32)>

In [7]:
tokenizer.detokenize([450, 4996, 12246, 1701, 29916, 29889])

'The quick bron fox.'

In [8]:
import tensorflow as tf

batch_size = 128
sequence_length = 256
suffix = np.array([tokenizer.token_to_id("<|endoftext|>")])

def read_file(filename):
    ds = tf.data.TextLineDataset(filename)
    ds = ds.map(lambda x: tf.strings.regex_replace(x, r"\\n", "\n"))
    ds = ds.map(tokenizer, num_parallel_calls=8)
    return ds.map(lambda x: tf.concat([x, suffix], -1))

files = [str(file) for file in extract_dir.glob("*.txt")]
ds = tf.data.Dataset.from_tensor_slices(files)
ds = ds.interleave(read_file, cycle_length=32, num_parallel_calls=32)
ds = ds.rebatch(sequence_length + 1, drop_remainder=True)
ds = ds.map(lambda x: (x[:-1], x[1:]))
ds = ds.batch(batch_size).prefetch(8)

In [9]:
num_batches = 29373
num_val_batches = 500
num_train_batches = num_batches - num_val_batches
val_ds = ds.take(num_val_batches).repeat()
train_ds = ds.skip(num_val_batches).repeat()

In [ ]:
from keras import layers

